<a href="https://colab.research.google.com/github/reneSalmon/ElasticSearch_Recommendation_System/blob/main/SemanticSearch_combined_with_ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## Import modules and packages

In [8]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [7]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Import libraries

In [13]:
import pandas as pd
import numpy as np
import json
import os
import uuid


from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

import opendatasets as od

## Import data

In [18]:
od.download(
    "https://www.kaggle.com/datasets/madhab/jobposts")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: renesalmon
Your Kaggle Key: ··········


100%|██████████| 14.0M/14.0M [00:00<00:00, 110MB/s] 

In [21]:
df  = pd.read_csv("/content/jobposts/data job posts.csv")
df

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18996,Technolinguistics NGO\r\n\r\n\r\nTITLE: Senio...,"Dec 28, 2015",Senior Creative UX/ UI Designer,Technolinguistics NGO,NaN,Full-time,NaN,NaN,NaN,Long-term,...,Competitive,"To apply for this position, please send your\r...",29 December 2015,28 January 2016,NaN,As a company Technolinguistics has a mandate t...,NaN,2015,12,False
18997,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18998,"""Coca-Cola Hellenic Bottling Company Armenia"" ...","Dec 30, 2015",Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,Full-time,All interested professionals.,NaN,ASAP,Long-term with a probation period of 3 months.,...,NaN,All interested candidates are kindly requested...,30 December 2015,20 January 2016,NaN,NaN,NaN,2015,12,False
18999,San Lazzaro LLC\r\n\r\n\r\nTITLE: Head of O...,"Dec 30, 2015",Head of Online Sales Department,San Lazzaro LLC,NaN,NaN,NaN,NaN,NaN,Long-term,...,Highly competitive,Interested candidates can send their CVs to:\r...,30 December 2015,29 January 2016,NaN,San Lazzaro LLC works with several internation...,NaN,2015,12,False


In [22]:
df.shape

(19001, 24)

#Data preprocessing

In [28]:
df = df.head(5000)
df.shape

(5000, 24)

In [37]:
#Drop rows where at least one element is missing
df_1 = df.dropna()
df_1.shape

(0, 24)

In [39]:
#Drop only rows where all elements are missing.
df_2 = df.dropna(how="all")
df_2.shape

(5000, 24)

#Create tokens

In [24]:
#Class to convert sentences into token

class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

In [43]:
token_instance = Tokenizer()

In [44]:
df['vector'] = df['jobpost'].progress_apply(token_instance.get_token)

  0%|          | 0/5000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
PUT posting 
{
   "settings":{
      "index":{
         "number_of_shards":1,
         "number_of_replicas":0,
         "knn":{
            "algo_param":{
               "ef_search":40,
               "ef_construction":40,
               "m":"4"
            }
         }
      },
      "knn":true
   },
   "mappings":{
      "properties":{
         "vector":{
            "type":"knn_vector",
            "dimension":384
         },
         "title":{
            "type":"text",
            "fields":{
               "keyword":{
                  "type":"keyword",
                   "index":false
               }
            }
         },
         "company":{
            "type":"keyword",
            "index":false
         },
         "location":{
            "type":"keyword",
            "index":false
         },
         "salary":{
            "type":"keyword",
            "index":false
         },
         "job_description":{
            "type":"keyword",
            "index":false
         }
      }
   }
}

SyntaxError: ignored

In [46]:
PUT posting1 
{
   "settings":{
      "index":{
         "number_of_shards":20,
         "number_of_replicas":1,
         "knn":{
            "algo_param":{
               "ef_search":40,
               "ef_construction":40,
               "m":"4"
            }
         }
      },
      "knn":true
   },
   "mappings":{
      "properties":{
         "vector":{
            "type":"knn_vector",
            "dimension":384
         }
      }
   }
}

SyntaxError: ignored

In [ ]:
elk_data = df.to_dict("records")

In [ ]:
es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD))
es.ping()

In [ ]:
for x in elk_data:
    try:
        _={
           "title":x.get("Title", ""),
            "company":x.get("Company", ""),
            "location":x.get("Location", ""),
            "salary":x.get("Salary", ""),
            "vector":x.get("vector", ""),
            "job_description":x.get("JobDescription", ""),

        }
        es.index(index='posting1',body=_)
    except Exception as e:pass